In [17]:
using Knet

In [35]:
head(t::Tuple) = tuple(t[1])

head (generic function with 1 method)

In [45]:
function viewblocks{T <: NTuple}(data::AbstractArray, shapes::AbstractVector{T})
    starts = cumsum(vcat([1], prod.(shapes)))
    [reshape(view(data, starts[i]:(starts[i+1] - 1)), shapes[i]) for i in 1:length(shapes)]
end

viewblocks (generic function with 3 methods)

In [46]:
predict = (flat_weights, shapes, x) -> begin
    weights = viewblocks(flat_weights, shapes)
    num_weights = sum(length, weights)
    biases = viewblocks(@view(flat_weights[(num_weights + 1):end]), head.(shapes))
    y = x
    J = eye(length(x))
    for i in 1:(length(shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = y .>= 0
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(w * y, w * J)
end

loss = (w, shapes, x, y) -> sumabs2(y - predict(w, shapes, x)) / size(y, 2)
lossgradient = grad(loss)

x = [1.0]
shapes = [(2, 1), (1, 2)]
w = [0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 3.0]

7-element Array{Float64,1}:
 0.2
 0.3
 0.4
 0.5
 1.0
 2.0
 3.0

In [47]:
predict(w, shapes, x)

2×1 Array{Float64,2}:
 1.63
 0.23

In [48]:
train_data = [([1], [1, 1]), ([-1], [1, -1])]

2-element Array{Tuple{Array{Int64,1},Array{Int64,1}},1}:
 ([1],[1,1])  
 ([-1],[1,-1])

In [49]:
lossgradient(w, shapes, [1], [1, 1])

LoadError: MethodError: no method matching view(::AutoGrad.Rec{Array{Float64,1}}, ::UnitRange{Int64})[0m
Closest candidates are:
  view{T,N}([1m[31m::AbstractArray{T,N}[0m, ::Union{AbstractArray{T,N},Colon,Real}...) at subarray.jl:63
  view([1m[31m::AbstractArray{T,N}[0m, ::Union{AbstractArray{T,N},Colon,Real}) at subarray.jl:68
  view{N}([1m[31m::AbstractArray{T,N}[0m, ::Union{AbstractArray{T,N},Colon,Real}...) at subarray.jl:73[0m

In [42]:
function train(w, shapes, data, lr=0.1)
    for (x, y) in data
        dw = lossgradient(w, shapes, x, y)
        for i in 1:length(w)
            w[i] -= lr * dw[i]
        end
    end
    w
end

train (generic function with 2 methods)

In [43]:
train(w, shapes, train_data)
w

LoadError: MethodError: no method matching viewblocks(::AutoGrad.Rec{Array{Float64,1}}, ::Array{Tuple{Int64,Int64},1})[0m
Closest candidates are:
  viewblocks{T<:Tuple{Vararg{T,N}}}([1m[31m::AbstractArray{T,N}[0m, ::AbstractArray{T<:Tuple{Vararg{T,N}},1}) at In[8]:2[0m

In [9]:
f(x, w, shapes)

([0.4],
[0.1 0.15])

INFO: No GPU found, Knet using the CPU


In [14]:
grad(x -> f(x, w, shapes)[1][1])(x)

2-element Array{Float64,1}:
 0.1 
 0.15